<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_projects/blob/main/AIconnect_translation/aiconnect_kor_eng_translate_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers[sentencepiece] accelerate  sacremoses evaluate sacrebleu

In [ ]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline
    )

from datasets import (
    load_dataset,
    DatasetDict,
    load_metric
    )

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)

seed_everything(42)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Data/aiconnect_translate"

/content/drive/MyDrive/Data/aiconnect_translate


In [ ]:
batch_name = f'helsinki_nlp_ko_en_{random.randrange(99999999)}'
batch_name

'helsinki_nlp_ko_en_85822412'

In [ ]:
raw_dataset = load_dataset("csv", data_files='train.csv', split='train')
train_valid = raw_dataset.train_test_split(.2)
split_datasets = DatasetDict({
    'train': train_valid['train'],
    'valid': train_valid['test'],
})
split_datasets

DatasetDict({
    train: Dataset({
        features: ['sid', '한국어', '영어'],
        num_rows: 128000
    })
    valid: Dataset({
        features: ['sid', '한국어', '영어'],
        num_rows: 32000
    })
})

In [ ]:
print("원문: ", split_datasets['train'][0]['한국어'])
print("번역문: ", split_datasets['train'][0]['영어'])

원문:  농촌체험관광은 지역 주민의 능동적 참여를 유도하며 특히 노인층과 부녀 노동력을 적극 활용해야 한다.
번역문:  Rural experiential tourism induces active participation of local residents, and especially the elderly and women's labor force should be actively utilized.


In [ ]:
MODEL_NAME = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
max_input_length = 32
max_target_length = 32

def preprocess_function(examples):
  inputs = [ex for ex in examples['한국어']]
  targets = [ex for ex in examples['영어']]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # tokenizer setup for target data
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length= max_target_length, truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
BATCH_SIZE = 16
import torch,  gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
num_epochs = 7 # 3
args = Seq2SeqTrainingArguments(
    output_dir = batch_name,
    evaluation_strategy="epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=num_epochs,
    weight_decay=0.1,
    gradient_accumulation_steps = 8,
    predict_with_generate=True,
    fp16=True, # 고속화 loose한 정확도
    gradient_checkpointing=True # 메모리 절약 대신 느려짐
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



In [ ]:
# trainer.evaluate(max_length=max_target_length) # to check validate loss and blue before training

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,1.448200,1.261989,36.398789
2,1.211000,1.165470,38.478781
3,1.111700,1.142689,38.977341


TrainOutput(global_step=3000, training_loss=1.3372726236979167, metrics={'train_runtime': 7399.4136, 'train_samples_per_second': 51.896, 'train_steps_per_second': 0.405, 'total_flos': 3254243033088000.0, 'train_loss': 1.3372726236979167, 'epoch': 3.0})

In [ ]:
# trainer.evaluate(max_length=max_target_length) # check validation data evaluation after the training

In [ ]:


test_dataset = pd.read_csv('./test.csv')

test_dataset

,sid,한국어
0,0,장덕은 덕을 많이 쌓아야 마을에서 훌륭한 인물과 큰 부자인 장자(長者)가 나온다는 ...
1,1,"2011년 3월 현재 대표는 권미령이 맡아 이끌고 있으며, 회원 수는 약 2,500..."
2,2,"또 제천시 봉양읍 구곡리에 있는 구곡소교가 11.5m로 가장 넓고, 같은 구곡리에 ..."
3,3,관덕정이 정확히 언제 건립되었는지는 알 수 없으나 조선시대 정유재란이 발생한 즈음인...
4,4,따라서 소규모의 산체인 오름은 민간신앙의 대상이 되기도 하였다.
...,...,...
1995,1995,제주도는 산소 주변을 돌담으로 쌓아서 이를 '산담'이라 한다.
1996,1996,"이후 매년 회원이 증가하여 1991년에는 30명의 회원이 활동하였다가, 1992년에..."
1997,1997,하천 연변의 평야 지대에는 독안동·사룡동·당골·묘재·승지동 등의 자연 마을이 들어서...
1998,1998,출입구에 들어서서 동쪽으로 올라가면 산 정상에는 장대인 진남대(鎭南臺)가 있다.


In [ ]:
test = []
for text in (test_dataset['한국어']):
  test.append([text])

In [ ]:
test[:10]

[['장덕은 덕을 많이 쌓아야 마을에서 훌륭한 인물과 큰 부자인 장자(長者)가 나온다는 의미를 담고 있다.'],
 ['2011년 3월 현재 대표는 권미령이 맡아 이끌고 있으며, 회원 수는 약 2,500여 명이다.'],
 ['또 제천시 봉양읍 구곡리에 있는 구곡소교가 11.5m로 가장 넓고, 같은 구곡리에 있는 본동교가 4.5m로 가장 짧다.'],
 ['관덕정이 정확히 언제 건립되었는지는 알 수 없으나 조선시대 정유재란이 발생한 즈음인 1590년대 말로 추측된다.'],
 ['따라서 소규모의 산체인 오름은 민간신앙의 대상이 되기도 하였다.'],
 ['이 절에서 가장 오래된 것으로는 강원도 유형문화재 37호로 지정되어 있는 오층석탑이 있다.'],
 ['집필 및 편집은 성남문화원 부설 향토문화연구소 부소장인 조유전 을 비롯하여 국사편찬위원회 사료조사실장 이상태, 한국토지박물관 학예연구사 윤우준, 가천대학교 강사 서승갑·김진호 등 5명이 맡았다.'],
 ['이는 『요지연도』가 서왕모의 전설에만 국한된 것이 아니라, 도교적 이상세계의 다양한 모습을 폭넓게 수용하고 있음을 보여주는 예이다.'],
 ['장이 서는 곳은 생활의 중심지로서 자연스럽게 장터가 생기고 이로 인해 시장 터임을 나타내는 동네 이름이 생긴다.'],
 ['마룡리는 백월산의 줄기가 남동쪽으로 길게 뻗어 나온 마지막 부분의 산록에 위치하고 있다.']]

In [ ]:
translator  = pipeline("translation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
translator(test[0])

[{'translation_text': 'Jangdeok means that there must be a lot of virtue to produce great people and great rich Jangja from the village.'}]

In [ ]:
translated = []

for text in test:
  translated.append(translator(text))

In [ ]:
df = pd.DataFrame(translated)
final = df[0].apply(pd.Series)
final = final.reset_index()
final.columns =['sid','영어']
final

,sid,영어
0,0,Jangdeok means that there must be a lot of vir...
1,1,"As of March 2011, the representative is led by..."
2,2,"In addition, Gugok Sogyo Bridge in Gugok-ri, B..."
3,3,It is not known exactly when Gwandeokjeong Pav...
4,4,"Therefore, oreum, a small-scale mountain body,..."
...,...,...
1995,1995,"Jejudo Island is called ""Sandam"" by stacking a..."
1996,1996,"Since then, the number of members has increase..."
1997,1997,"Natural villages such as Dokan-dong, Saryong-d..."
1998,1998,When you enter the entrance and go to the east...


In [ ]:
final.to_csv ('./submit.csv', index=False)
print('Done')

Done
